In [ ]:
#1
SELECT city.Name AS Kota, country.Name AS Negara, 
city.Population 
FROM city, country  
WHERE city.CountryCode = country.Code 
ORDER BY city.population DESC 
LIMIT 10;

SELECT * FROM city;

#2
SELECT country.Name AS Negara, country.GNP, 
city.Name AS Kota, city.population
FROM country 
JOIN city
ON country.Code = city.CountryCode
WHERE country.Name = 'Netherlands' AND 
country.Capital = city.ID;


#3
SELECT c.Name AS Negara, cl.Percentage 
AS Persentase
FROM country c 
INNER JOIN countrylanguage cl
ON c.Code = cl.CountryCode
WHERE cl.language = 'Spanish'
ORDER BY cl.percentage DESC
LIMIT 10;



#4
SELECT country.Name AS Negara, country.GNP,
city.Name AS 'Ibu Kota', city.Population 
AS Populasi, countrylanguage.Language

FROM country 
JOIN city 
ON country.Code = city.CountryCode
JOIN countrylanguage 
ON country.Code = countrylanguage.CountryCode

WHERE country.Name = 'Indonesia'
AND countrylanguage.IsOfficial = 'T'
AND country.capital = city.ID;



#5
# Mengecek jumlah negara di North America
SELECT count(Name) 
FROM country 
WHERE continent = 'North America';

SELECT continent, count(Name) AS 'Jumlah Negara' 
FROM country
GROUP BY continent
HAVING count(Name) > (SELECT count(Name) 
FROM country WHERE continent = 
'North America');



#6
# Mengecek rata-rata GNP Europe
SELECT AVG(GNP) 
FROM country
WHERE continent = 'Europe';

SELECT name AS Negara, GNP 
FROM country
WHERE continent = 'Asia'
AND GNP > (SELECT AVG(GNP) FROM 
country WHERE continent = 'Europe')
ORDER BY GNP DESC;



#7
# Mengecek jumlah region unik di Asia
SELECT count(DISTINCT(region)) 
FROM country 
GROUP BY continent 
HAVING continent='Asia';

SELECT COUNT(DISTINCT(region)) AS 
'Jumlah Region Unik', continent 

FROM country
WHERE continent LIKE '%a'
GROUP BY continent 
HAVING COUNT(DISTINCT(region)) > 
(SELECT COUNT(DISTINCT(region)) 
FROM country 
GROUP BY continent 
HAVING continent = 'Asia');


In [ ]:
## SQL DAY 4

## COMMON TABLE EXPRESSION (CTE)

# CTE adalah sebuah query dalam SQL yang berupa sub-query 
  # yang ditulis secara terpisah, sehingga dapat dipergunakan 
  # kembali.
# Biasa digunakan untuk breakdown query yang complex.


# Syntax: 
# WITH nama_tabel_CTE AS (isi query nya) SELECT .. FROM ..

# Cara kerjanya, SQL akan menjalankan CTE terlebih dahulu, 
  # kemudian hasilnya disimpan dalam sebuah tabel sementara 
  # sesuai dengan nama CTE-nya.
# Kemudian, tabel CTE ini bisa digunakan berulang kali 
  # untuk keperluan pengambilan data atau join table. 

# Kelebihan:
	# - Cara penulisannya lebih rapi dan mudah dibaca.
	# - Dapat digunakan kembali. 
    
# Kekurangan:
	# - Running lebih lambat dari metode lainnya. 
    
USE sakila;
SHOW FULL TABLES;

SELECT * FROM film;

## Contoh
# Menampilkan film dengan rental duration yang lebih 
  # besar dari rata-rata rental duration secara keseluruhan.

# Pertama, cari dulu rata-rata rental duration 
  # secara keseluruhan
SELECT AVG(rental_duration)
FROM film;

# Kemudian, query di atas dijadikan sebagai sub-query 
  # untuk filtering
SELECT * FROM film
WHERE rental_duration > (SELECT AVG(rental_duration) 
FROM film);

## ==============================================================================================================

## Cara menggunakan CTE

## Membuat tabel CTE berisi average rental duration

# Cara 1
WITH avg_rental_duration AS 
(SELECT AVG(rental_duration) AS avg_rental 
FROM film)	# CTE

SELECT * FROM film
WHERE rental_duration > 
(SELECT avg_rental FROM avg_rental_duration);


# Cara 2
WITH avg_rental_duration AS
(SELECT AVG(rental_duration) AS avg_rental 
FROM film)

SELECT * 
FROM film F, avg_rental_duration T 
WHERE F.rental_duration > T.avg_rental;


USE WORLD; 

# Gunakan CTE untuk menampilkan benua dengan jumlah 
  # negara lebih dari jumlah negara di benua North 
  # America.

# Pertama, kita cari dulu jumlah negara di continent 
  # North America
SELECT COUNT(name) AS jumlah_negara
FROM country
WHERE continent = 'North America';

# Query di atas, kita gunakan sebagai CTE
WITH tabel_jumlah_negara AS
(SELECT COUNT(name) AS jumlah_negara_cte 
FROM country WHERE continent = 'North America')
                            
SELECT continent, COUNT(name) AS jumlah_negara
FROM country
GROUP BY continent
HAVING jumlah_negara > (SELECT jumlah_negara_cte 
                      FROM tabel_jumlah_negara);


# Tampilkan movies yang jumlah actornya di atas 
  # rata-rata.
  # Return nama film, jumlah aktor, diurutkan 
  # berdasarkan jumlah aktor terbanyak. 
  # Tampilkan 10 teratas!

WITH cte AS(
SELECT F.film_id, F.title, COUNT(FA.actor_id) AS jumlah_aktor
FROM film_actor FA
INNER JOIN film F ON FA.film_id = F.film_id
GROUP BY F.film_id, F.title
)

SELECT cte.title, cte.jumlah_aktor, 
AVG(jumlah_aktor) OVER() # menampilkan rata2 jumlah aktor
FROM cte 
WHERE cte.jumlah_aktor > (SELECT AVG(jumlah_aktor) 
FROM cte)
ORDER BY cte.actor_count DESC 
LIMIT 10;

## ==============================================================================================================

## WINDOW FUNCTION

# Kalau menggunakan GROUP BY, terdapat key column
  # yang isinya adalah distinct value (nilai unik), 
  # dan kolom lainnya adalah agregasi.
# GROUP BY menyebabkan berkurangnya jumlah baris.
# Baris yang ditampilkan sesuai dengan jumlah 
  # distinct value pada key column. 

# Tapi, dengan WINDOW FUNCTION, kita dapat melakukan 
  # agregasi dengan tetap mempertahankan jumlah 
  # baris sebagaimana adanya. 
# Semua value tetap pada row-nya. 

USE sakila;

# Contoh dengan menggunakan GROUP BY

# Menampilkan rata-rata rental duration
  # berdasarkan rating
SELECT rating, AVG(rental_duration)
FROM film
GROUP BY rating;

## ==============================================================================================================

## OVER PARTITION

# Cara kerja mirip dengan GROUP BY. 
  # Yang membedakan adalah jumlah baris 
  # yang dikembalikan pada output 
  # (mengembalikan sesuai dengan jumlah 
  # baris yang ada).

SELECT * FROM film;

# Menggunakan over
# Menghasilkan 1 nilai average saja
SELECT film_id, title, rating, rental_rate,
AVG(rental_rate) OVER () AS avg_rental_rate, # Ini rata-rata rental_rate secara keseluruhan
AVG(rental_rate) OVER () - rental_rate 
AS selisih_avg_dikurang_rental_rate	# Ini selisih rata-rata rental_rate keseluruhan dengan rental_rate tiap baris
FROM film;


# Menggunakan OVER PARTITION
# Seperti GROUP BY
SELECT film_id, title, rating, rental_rate,
	AVG(rental_rate) OVER (PARTITION BY rating) 
    AS avg_rental_rate_by_rating,
	AVG(rental_rate) OVER (PARTITION BY rating) - 
    rental_rate AS selisih
FROM film;


# Gunakan OVER PARTITION untuk menampilkan 
  # nama film, category, length, dan rata-rata 
  # durasi film berdasarkan category.

SHOW FULL TABLES;

# Mengambil data dari base table

SELECT A.film_id, A.title, C.name, A.length, 
	AVG(A.length) OVER (PARTITION BY B.category_id) 
  AS avg_durasi
FROM film A
JOIN film_category B
ON A.film_id = B.film_id
JOIN category C
ON B.category_id= C.category_id;

# Mengambil data dari VIEW
SELECT * FROM film_list;

SELECT title, category, length,
	AVG(length) OVER (PARTITION BY category) 
  AS avg_length_by_category
FROM film_list;

# Menarik data employees, umur, salary terakhir, 
  # department, avg salary per department
SELECT E.first_name, E.last_name, 
YEAR(NOW()) - YEAR(E.birth_date) AS age, 
S.salary, D.dept_name, AVG(S.salary) 

OVER(PARTITION BY D.dept_name) AS Avg_Salary
FROM employees E

JOIN salaries S ON E.emp_no = S.emp_no
JOIN dept_emp DE ON S.emp_no = DE.emp_no
JOIN departments D ON DE.dept_no = D.dept_no

WHERE S.to_date = (SELECT MAX(S2.to_date) 
FROM salaries S2 WHERE S2.emp_no = E.emp_no);

## ==============================================================================================================

## NON-AGGREGATE FUNCTION

## ROW_NUMBER

# Membuat kolom baru berisikan nomor baris.
# Mirip dengan index baris, tapi ROW_NUMBER 
  # ini disimpan dalam sebuah kolom. 

# Contoh

# Menampilkan ROW NUMBER secara keseluruhan
SELECT ROW_NUMBER () OVER () AS Nomor,
	title, rating, rental_duration
FROM film;

# Menampilkan ROW NUMBER berdasarkan rating-nya
SELECT ROW_NUMBER () OVER (PARTITION BY rating) 
AS Nomor_by_rating,
title, rating, rental_duration
FROM film;

## ==============================================================================================================

## RANK DAN DENSE_RANK

# ROW_NUMBER menghitung baris data dalam angka 1 
  # sampai n dari urutan terkecil.
# RANK dan DENSE_RANK menghitung urutan berdasarkan 
  # value yang ingin kita ukur dan bisa dari 
  # tertinggi ke terendah.


# Contoh

# Tampilkan ranking untuk category rating 
  # dengan total film paling banyak 

SELECT rating, COUNT(film_id) AS Jumlah_Film,
	RANK () OVER (ORDER BY COUNT(film_id) DESC) 
  AS Ranking
FROM film
GROUP BY rating;

# ORDER BY seperti biasa, tanpa diketahui 
  # urutan atau ranking ke berapanya
SELECT rating, COUNT(film_id) AS Jumlah_Film
FROM film
GROUP BY rating
ORDER BY Jumlah_Film DESC;

# RANK --> dari peringkat 1 langsung lanjut 
  # ke peringkat 204 (karena ada 203 film 
  # yang berdurasi sewa 3 hari) --> WITH GAPS
SELECT film_id, title, rental_duration, 
	RANK () OVER (ORDER BY rental_duration) 
  AS Ranking
FROM film;

# DENSE_RANK --> dari peringkat 1 lanjut ke 
  # peringkat 2, meskipun ada 203 film 
  # berdurasi 3 hari --> WITHOUT GAPS
SELECT film_id, title, rental_duration, 
	DENSE_RANK () OVER (ORDER BY rental_duration) 
  AS Ranking
FROM film;

# Employees 
SELECT ROW_NUMBER() OVER (PARTITION BY 
    D.dept_name ORDER BY S.salary DESC) 
AS emp_row_no, CONCAT(E.first_name, ' ',
                  E.last_name) AS name, 
D.dept_name, S.salary,

RANK() OVER(PARTITION BY D.dept_name 
		ORDER BY S.salary DESC) AS emp_rank,
DENSE_RANK() OVER(PARTITION BY D.dept_name 
    ORDER BY S.salary DESC) 
    AS emp_dense_rank

FROM employees E
JOIN salaries S ON E.emp_no = S.emp_no
JOIN dept_emp DE ON S.emp_no = DE.emp_no
JOIN departments D ON DE.dept_no = D.dept_no

WHERE S.to_date = (SELECT MAX(S2.to_date) 
FROM salaries S2 WHERE S2.emp_no = E.emp_no);


## ==============================================================================================================

## NTILE ()

# Mengelompokkan data dari terkecil ke terbesar. 
# Jumlah kelompoknya disesuaikan dengan persentase 
  # pembagian yang kita input-kan. 
# NTILE(4) --> Artinya data akan dibagi menjadi 4 
  # bagian/kelompok dengan tiap kelompoknya memiliki 
  # jumlah data yang sama. 

# Contoh

SELECT title, rating,
	NTILE(4) OVER () AS Quartile,
    NTILE(10) OVER () AS Percentile,
    ROW_NUMBER() OVER () AS Nomor
FROM film;

## ==============================================================================================================

## SLIDING WINDOWS

# Digunakan untuk menghitung angka agregat yang 
  # bersifat bergerak atau kumulatif. 
# Bisa digunakan untuk menghitung moving average, 
  # cumulative sum, dll. 

# Syntax:
# OVER (ROWS BETWEEN tipe_sliding1 AND tipe_sliding2)

# TIPE SLIDING:
	# - CURRENT ROW --> Row yang aktif
    # - FOLLOWING --> Row setelah
    # - PRECEDING --> Row sebelum
    # - UNBOUNDED PRECEDING --> Row pertama
    # - UNBOUNDED FOLLOWING --> Row terakhir

# Contoh

# Menghitung cummulative sum dari total film berdasarkan rating

WITH tabel_jumlah_film_per_rating AS
	(SELECT rating, COUNT(film_id) AS jumlah_film
    FROM film
    GROUP BY rating)
    
SELECT rating, jumlah_film, 
	SUM(jumlah_film) OVER (ORDER BY jumlah_film 
  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
  AS cum_sum_jumlah_film,
  AVG(jumlah_film) OVER (ORDER BY jumlah_film ROWS 
  BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
  AS moving_avg_jumlah_film
FROM tabel_jumlah_film_per_rating;

## ==============================================================================================================

## LAG
  # returns the value to the previous rows in 
  # a sorted and partitioned result set.
# Misal untuk membandingkan dengan sales 
  # periode sebelumnya.
  
select * from payment;

SELECT customer_id, CONCAT(MONTH(payment_date), 
      '-', YEAR(payment_date)) AS date, amount,
LAG(amount, 1) OVER (PARTITION BY customer_id 
              ORDER BY payment_date) prev_rent
FROM payment;


## LEAD
  # exact opposite operation of lag() function. 
  # Lead() function will return the values ahead, 
  # in the partitioned and sorted result set.
# Bisa digunakan untuk target sales tahun 
  # berikutnya misalnya.

SELECT customer_id, CONCAT(MONTH(payment_date), 
      '-', YEAR(payment_date)) AS date, amount,
LEAD(amount, 1) OVER (PARTITION BY customer_id 
              ORDER BY payment_date) prev_rent
FROM payment;